In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from bs4 import BeautifulSoup
import pandas as pd
import time

# Setting up the Chrome WebDriver
options = webdriver.ChromeOptions()
options.headless = True  # Set headless mode to True to run without opening a browser window
driver = webdriver.Chrome(options=options)

data_inc = {'car_name': [], 'car_info': [], 'car_year_hand': [], 'car_price': [],'car_tag': []}
for i in range(1, 201):
    url = f"https://www.yad2.co.il/vehicles/cars?page={i}"
    driver.get(url)
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))  # Simple condition
    except TimeoutException:
        print("Timed out waiting for page to load: ", url)
        continue
    
    # Scroll to the bottom of the page (if needed)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Allow extra time for any lazy-loaded elements
    
    # Extract HTML content
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, "html.parser")
    car_boxes = soup.find_all('div', class_='feed-item-base_feedItemBox__5WVY1 scale-up-on-hover_scaleUpOnHover__QSVDw')
    for box in car_boxes:
        car_name = box.find('span', class_='feed-item-info_heading__k5pVC').get_text(strip=True)
        car_info = box.find('span', class_='feed-item-info_marketingText__eNE4R').get_text(strip=True)
        car_year_hand = box.find('span', class_='feed-item-info_yearAndHandBox___JLbc').get_text(strip=True)
        car_price = box.find('span', class_='price_price__xQt90').get_text(strip=True)
        car_tags = box.find('div', class_='tag-list_upgradeItemFlagBox__qPadL')
        if car_tags:
            car_tags_texts = [tag.get_text(strip=True) for tag in car_tags.find_all('span', class_='tag-list_tag__EX6gE flag_baseFlag__PIHA5 flag_grayFlag__1s_oj')]
        else:
            car_tags_texts = ['']
        # data_inc['car_tag'].extend(car_tags_texts)
        data_inc['car_name'].append(car_name)
        data_inc['car_info'].append(car_info)
        data_inc['car_year_hand'].append(car_year_hand)
        data_inc['car_price'].append(car_price)
        data_inc['car_tag'].append(','.join(car_tags_texts))
        # if title_element and date_element:
        #     title = title_element.get_text(strip=True)
        #     date = date_element.get_text(strip=True)
        #     link = title_element.find('a')['href'] if title_element.find('a') else None
        #     data_inc['title'].append(title)
        #     data_inc['date'].append(date)
        #     data_inc['link'].append(link)
# Close the driver after the loop
driver.quit()# Display the first few rows of the DataFrame


KeyboardInterrupt: 

In [13]:
df_car_prices = pd.DataFrame(data_inc)
df_car_prices

,car_name,car_info,car_year_hand,car_price,car_tag
0,טויוטה קורולה הייבריד,Limited סדאן אוט' 1.8 (98 כ''ס),2021 • יד 2,"129,600 ₪","בקרת שמירת מרחק,גלגלי מגנזיום,היברידי"
1,פולקסווגן ג'טה,"Trendline אוט' 1.2 (105 כ""ס)",2015 • יד 2,"24,000 ₪","טסט בתוקף,מערכת מולטימדיה,בהזדמנות"
2,קיה פיקנטו,"EX אוט' 1.2 (85 כ""ס)",2015 • יד 1,"39,000 ₪","גלגלי מגנזיום,טסט בתוקף,יד ראשונה"
3,לנד רובר דיפנדר,4X4 TD5 110 סגור ידני דיזל 2.5 (138 כ''ס),2001 • יד 5,"145,000 ₪","מיכל דלק גדול,שטח קשוח,טסט בתוקף"
4,פיג'ו 2008,Premium אוט' 1.2 (130 כ''ס),2020 • יד 1,"90,000 ₪","בקרת שיוט אדפטיבית,בקרת שמירת מרחק,גלגלי מגנזיום"
...,...,...,...,...,...
1195,סקודה אוקטביה,Ambition אוט' 1.5 (150 כ''ס),2020 • יד 1,"87,000 ₪","טסט בתוקף,יד ראשונה,אפשרויות למימון"
1196,ניסאן קשקאי,Acenta אוט' 1.3 (159 כ''ס),2021 • יד 1,"95,000 ₪","בקרת שמירת מרחק,חסכוני,גלגלי מגנזיום"
1197,סקודה קארוק,Ambition אוט' 1.5 (150 כ''ס),2021 • יד 1,"115,000 ₪","בקרת שיוט אדפטיבית,בקרת שמירת מרחק,גלגלי מגנזיום"
1198,סקודה אוקטביה RS,"RS אוט' 2.0 (220 כ""ס)",2017 • יד 4,"90,000 ₪","חסכוני,גלגלי מגנזיום,סאן-רוף"


In [14]:
df_car_prices.to_csv('yad2_cars_data_2.csv', index=False)



In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from bs4 import BeautifulSoup
import pandas as pd
import time

# Setting up the Chrome WebDriver
options = webdriver.ChromeOptions()
options.headless = True  # Set headless mode to True to run without opening a browser window
driver = webdriver.Chrome(options=options)

data_inc = {'car_name': [], 'car_info': [], 'car_year_hand': [], 'car_price': [],'car_tag': []}
for i in range(1, 201):
    url = f"https://www.yad2.co.il/vehicles/cars?page={i}"
    driver.get(url)
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))  # Simple condition
    except TimeoutException:
        print("Timed out waiting for page to load: ", url)
        continue
    
    # Scroll to the bottom of the page (if needed)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Allow extra time for any lazy-loaded elements
    
    # Extract HTML content
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, "html.parser")
    car_boxes = soup.find_all('div', class_='feed-item-base_feedItemBox__5WVY1 scale-up-on-hover_scaleUpOnHover__QSVDw')
    for box in car_boxes:
        car_name = box.find('span', class_='feed-item-info_heading__k5pVC').get_text(strip=True)
        car_info = box.find('span', class_='feed-item-info_marketingText__eNE4R').get_text(strip=True)
        car_year_hand = box.find('span', class_='feed-item-info_yearAndHandBox___JLbc').get_text(strip=True)
        car_price = box.find('span', class_='price_price__xQt90').get_text(strip=True)
        car_tags = box.find('div', class_='tag-list_upgradeItemFlagBox__qPadL')
        if car_tags:
            car_tags_texts = [tag.get_text(strip=True) for tag in car_tags.find_all('span', class_='tag-list_tag__EX6gE flag_baseFlag__PIHA5 flag_grayFlag__1s_oj')]
        else:
            car_tags_texts = ['']
        # data_inc['car_tag'].extend(car_tags_texts)
        data_inc['car_name'].append(car_name)
        data_inc['car_info'].append(car_info)
        data_inc['car_year_hand'].append(car_year_hand)
        data_inc['car_price'].append(car_price)
        data_inc['car_tag'].append(','.join(car_tags_texts))
        # if title_element and date_element:
        #     title = title_element.get_text(strip=True)
        #     date = date_element.get_text(strip=True)
        #     link = title_element.find('a')['href'] if title_element.find('a') else None
        #     data_inc['title'].append(title)
        #     data_inc['date'].append(date)
        #     data_inc['link'].append(link)
# Close the driver after the loop
driver.quit()# Display the first few rows of the DataFrame


KeyboardInterrupt: 

In [24]:
df_car_prices = pd.DataFrame(data_inc)
df_car_prices.to_csv('yad2_cars_data_4.csv', index=False)



In [32]:
df1 = pd.read_csv('yad2_cars_data_2.csv')
df2 = pd.read_csv('yad2_cars_data_3.csv')
df3 = pd.read_csv('yad2_cars_data_4.csv')
df4 = pd.concat([df1, df2,df3])

df4

,car_name,car_info,car_year_hand,car_price,car_tag
0,טויוטה קורולה הייבריד,Limited סדאן אוט' 1.8 (98 כ''ס),2021 • יד 2,"129,600 ₪","בקרת שמירת מרחק,גלגלי מגנזיום,היברידי"
1,פולקסווגן ג'טה,"Trendline אוט' 1.2 (105 כ""ס)",2015 • יד 2,"24,000 ₪","טסט בתוקף,מערכת מולטימדיה,בהזדמנות"
2,קיה פיקנטו,"EX אוט' 1.2 (85 כ""ס)",2015 • יד 1,"39,000 ₪","גלגלי מגנזיום,טסט בתוקף,יד ראשונה"
3,לנד רובר דיפנדר,4X4 TD5 110 סגור ידני דיזל 2.5 (138 כ''ס),2001 • יד 5,"145,000 ₪","מיכל דלק גדול,שטח קשוח,טסט בתוקף"
4,פיג'ו 2008,Premium אוט' 1.2 (130 כ''ס),2020 • יד 1,"90,000 ₪","בקרת שיוט אדפטיבית,בקרת שמירת מרחק,גלגלי מגנזיום"
...,...,...,...,...,...
1875,טויוטה RAV4 ארוך,Extreme אוט' 2.0 (175 כ''ס),2020 • יד 1,לא צוין מחיר,"בקרת שמירת מרחק,חסכוני,גלגלי מגנזיום"
1876,אאודי A5,"Sportback Design Luxury אוט' 2.0 (150 כ""ס)",2019 • יד 2,"146,800 ₪","בקרת שיוט אדפטיבית,בקרת שמירת מרחק,גלגלי מגנזיום"
1877,שברולט טראקס,"LT אוט' 1.8 (140 כ""ס)",2015 • יד 2,"32,000 ₪","חסכוני,גלגלי מגנזיום,טסט בתוקף"
1878,ב.מ.וו סדרה 3,330E Sport הייבריד אוט' 2.0 (184 כ''ס),2019 • יד 1,לא צוין מחיר,"בקרת שמירת מרחק,גלגלי מגנזיום,היברידי"


In [38]:
df4.drop_duplicates(inplace=True)
df4




,car_name,car_info,car_year_hand,car_price,car_tag,car_name_en,car_info_en
0,טויוטה קורולה הייבריד,Limited סדאן אוט' 1.8 (98 כ''ס),2021 • יד 2,"129,600 ₪","בקרת שמירת מרחק,גלגלי מגנזיום,היברידי",Toyota Corolla Hybrid,Limited Sedan Out 1.8 (98 HP)
1,פולקסווגן ג'טה,"Trendline אוט' 1.2 (105 כ""ס)",2015 • יד 2,"24,000 ₪","טסט בתוקף,מערכת מולטימדיה,בהזדמנות",Volkswagen Jetta,Trendline automatic 1.2 (105 hp)
2,קיה פיקנטו,"EX אוט' 1.2 (85 כ""ס)",2015 • יד 1,"39,000 ₪","גלגלי מגנזיום,טסט בתוקף,יד ראשונה",Kia Picanto,EX auto 1.2 (85 hp)
3,לנד רובר דיפנדר,4X4 TD5 110 סגור ידני דיזל 2.5 (138 כ''ס),2001 • יד 5,"145,000 ₪","מיכל דלק גדול,שטח קשוח,טסט בתוקף",Land Rover Defender,4X4 TD5 110 closed manual diesel 2.5 (138 hp)
4,פיג'ו 2008,Premium אוט' 1.2 (130 כ''ס),2020 • יד 1,"90,000 ₪","בקרת שיוט אדפטיבית,בקרת שמירת מרחק,גלגלי מגנזיום",Peugeot 2008,Premium auto 1.2 (130 hp)
...,...,...,...,...,...,...,...
1875,טויוטה RAV4 ארוך,Extreme אוט' 2.0 (175 כ''ס),2020 • יד 1,לא צוין מחיר,"בקרת שמירת מרחק,חסכוני,גלגלי מגנזיום",Toyota RAV4 is long,Extreme Auto 2.0 (175 hp)
1876,אאודי A5,"Sportback Design Luxury אוט' 2.0 (150 כ""ס)",2019 • יד 2,"146,800 ₪","בקרת שיוט אדפטיבית,בקרת שמירת מרחק,גלגלי מגנזיום",Audi A5,Sportback Design Luxury Auto 2.0 (150 HP)
1877,שברולט טראקס,"LT אוט' 1.8 (140 כ""ס)",2015 • יד 2,"32,000 ₪","חסכוני,גלגלי מגנזיום,טסט בתוקף",Chevrolet Trax,LT auto 1.8 (140 hp)
1878,ב.מ.וו סדרה 3,330E Sport הייבריד אוט' 2.0 (184 כ''ס),2019 • יד 1,לא צוין מחיר,"בקרת שמירת מרחק,גלגלי מגנזיום,היברידי",BMW 3 Series,330E Sport Hybrid OT 2.0 (184 HP)


In [43]:
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='auto', target='en')

# Function to translate text using deep_translator
def deep_translate_text(text, target='en'):
    return translator.translate(text)
   
# # Apply the translation function to the DataFrame
# df4['car_name_en'] = df4['car_name'].apply(deep_translate_text)
# df4['car_info_en'] = df4['car_info'].apply(deep_translate_text)


df4

,car_name,car_info,car_year_hand,car_price,car_tag,car_name_en,car_info_en,car_tags_en
0,טויוטה קורולה הייבריד,Limited סדאן אוט' 1.8 (98 כ''ס),2021 • יד 2,"129,600 ₪","בקרת שמירת מרחק,גלגלי מגנזיום,היברידי",Toyota Corolla Hybrid,Limited Sedan Out 1.8 (98 HP),"Distance control, magnesium wheels, hybrid"
1,פולקסווגן ג'טה,"Trendline אוט' 1.2 (105 כ""ס)",2015 • יד 2,"24,000 ₪","טסט בתוקף,מערכת מולטימדיה,בהזדמנות",Volkswagen Jetta,Trendline automatic 1.2 (105 hp),"Valid test, multimedia system, on occasion"
2,קיה פיקנטו,"EX אוט' 1.2 (85 כ""ס)",2015 • יד 1,"39,000 ₪","גלגלי מגנזיום,טסט בתוקף,יד ראשונה",Kia Picanto,EX auto 1.2 (85 hp),"Magnesium wheels, fully tested, first hand"
3,לנד רובר דיפנדר,4X4 TD5 110 סגור ידני דיזל 2.5 (138 כ''ס),2001 • יד 5,"145,000 ₪","מיכל דלק גדול,שטח קשוח,טסט בתוקף",Land Rover Defender,4X4 TD5 110 closed manual diesel 2.5 (138 hp),"Large fuel tank, tough terrain, fully tested"
4,פיג'ו 2008,Premium אוט' 1.2 (130 כ''ס),2020 • יד 1,"90,000 ₪","בקרת שיוט אדפטיבית,בקרת שמירת מרחק,גלגלי מגנזיום",Peugeot 2008,Premium auto 1.2 (130 hp),"Adaptive cruise control, cruise control, magne..."
...,...,...,...,...,...,...,...,...
1875,טויוטה RAV4 ארוך,Extreme אוט' 2.0 (175 כ''ס),2020 • יד 1,לא צוין מחיר,"בקרת שמירת מרחק,חסכוני,גלגלי מגנזיום",Toyota RAV4 is long,Extreme Auto 2.0 (175 hp),"Distance control, economical, magnesium wheels"
1876,אאודי A5,"Sportback Design Luxury אוט' 2.0 (150 כ""ס)",2019 • יד 2,"146,800 ₪","בקרת שיוט אדפטיבית,בקרת שמירת מרחק,גלגלי מגנזיום",Audi A5,Sportback Design Luxury Auto 2.0 (150 HP),"Adaptive cruise control, cruise control, magne..."
1877,שברולט טראקס,"LT אוט' 1.8 (140 כ""ס)",2015 • יד 2,"32,000 ₪","חסכוני,גלגלי מגנזיום,טסט בתוקף",Chevrolet Trax,LT auto 1.8 (140 hp),"Economical, magnesium wheels, fully tested"
1878,ב.מ.וו סדרה 3,330E Sport הייבריד אוט' 2.0 (184 כ''ס),2019 • יד 1,לא צוין מחיר,"בקרת שמירת מרחק,גלגלי מגנזיום,היברידי",BMW 3 Series,330E Sport Hybrid OT 2.0 (184 HP),"Distance control, magnesium wheels, hybrid"


In [ ]:
<div class="tag-list_upgradeItemFlagBox__qPadL"><span class="tag-list_tag__EX6gE flag_baseFlag__PIHA5 flag_grayFlag__1s_oj"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Test in force</font></font></span><span class="tag-list_tag__EX6gE flag_baseFlag__PIHA5 flag_grayFlag__1s_oj"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">first hand</font></font></span><span class="tag-list_tag__EX6gE flag_baseFlag__PIHA5 flag_grayFlag__1s_oj"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">without accidents</font></font></span></div>

SyntaxError: invalid syntax (1866410033.py, line 1)

In [41]:
df4['car_tag'].unique()



array(['בקרת שמירת מרחק,גלגלי מגנזיום,היברידי',
       'טסט בתוקף,מערכת מולטימדיה,בהזדמנות',
       'גלגלי מגנזיום,טסט בתוקף,יד ראשונה',
       'מיכל דלק גדול,שטח קשוח,טסט בתוקף',
       'בקרת שיוט אדפטיבית,בקרת שמירת מרחק,גלגלי מגנזיום',
       'חסכוני,תא מטען גדול,טסט בתוקף',
       'בקרת שיוט אדפטיבית,בקרת שמירת מרחק,חסכוני',
       'בקרת שיוט אדפטיבית,גלגלי מגנזיום,היברידי',
       'בקרת שמירת מרחק,חסכוני,גלגלי מגנזיום',
       'חסכוני,גלגלי מגנזיום,טסט בתוקף', 'שטח קשוח,אבזור אקסטרה,בהזדמנות',
       'בקרת שמירת מרחק,גלגלי מגנזיום,טסט בתוקף',
       'מיכל דלק גדול,גלגלי מגנזיום,מנוע חזק',
       'מיכל דלק גדול,גלגלי מגנזיום,סאן-רוף',
       'בקרת שיוט אדפטיבית,בקרת שמירת מרחק,היברידי',
       'חסכוני,גלגלי מגנזיום,חשמלי', 'אפשרויות למימון,חבל לפספס,בהזדמנות',
       nan, 'בקרת שמירת מרחק,חסכוני,טסט בתוקף',
       'בקרת שמירת מרחק,גלגלי מגנזיום,אפשרויות למימון',
       'טסט בתוקף,יד ראשונה,אפשרויות למימון',
       'בקרת שיוט אדפטיבית,בקרת שמירת מרחק,באחריות יבואן',
       'בקרת שיו

In [40]:
df4

,car_name,car_info,car_year_hand,car_price,car_tag,car_name_en,car_info_en
0,טויוטה קורולה הייבריד,Limited סדאן אוט' 1.8 (98 כ''ס),2021 • יד 2,"129,600 ₪","בקרת שמירת מרחק,גלגלי מגנזיום,היברידי",Toyota Corolla Hybrid,Limited Sedan Out 1.8 (98 HP)
1,פולקסווגן ג'טה,"Trendline אוט' 1.2 (105 כ""ס)",2015 • יד 2,"24,000 ₪","טסט בתוקף,מערכת מולטימדיה,בהזדמנות",Volkswagen Jetta,Trendline automatic 1.2 (105 hp)
2,קיה פיקנטו,"EX אוט' 1.2 (85 כ""ס)",2015 • יד 1,"39,000 ₪","גלגלי מגנזיום,טסט בתוקף,יד ראשונה",Kia Picanto,EX auto 1.2 (85 hp)
3,לנד רובר דיפנדר,4X4 TD5 110 סגור ידני דיזל 2.5 (138 כ''ס),2001 • יד 5,"145,000 ₪","מיכל דלק גדול,שטח קשוח,טסט בתוקף",Land Rover Defender,4X4 TD5 110 closed manual diesel 2.5 (138 hp)
4,פיג'ו 2008,Premium אוט' 1.2 (130 כ''ס),2020 • יד 1,"90,000 ₪","בקרת שיוט אדפטיבית,בקרת שמירת מרחק,גלגלי מגנזיום",Peugeot 2008,Premium auto 1.2 (130 hp)
...,...,...,...,...,...,...,...
1875,טויוטה RAV4 ארוך,Extreme אוט' 2.0 (175 כ''ס),2020 • יד 1,לא צוין מחיר,"בקרת שמירת מרחק,חסכוני,גלגלי מגנזיום",Toyota RAV4 is long,Extreme Auto 2.0 (175 hp)
1876,אאודי A5,"Sportback Design Luxury אוט' 2.0 (150 כ""ס)",2019 • יד 2,"146,800 ₪","בקרת שיוט אדפטיבית,בקרת שמירת מרחק,גלגלי מגנזיום",Audi A5,Sportback Design Luxury Auto 2.0 (150 HP)
1877,שברולט טראקס,"LT אוט' 1.8 (140 כ""ס)",2015 • יד 2,"32,000 ₪","חסכוני,גלגלי מגנזיום,טסט בתוקף",Chevrolet Trax,LT auto 1.8 (140 hp)
1878,ב.מ.וו סדרה 3,330E Sport הייבריד אוט' 2.0 (184 כ''ס),2019 • יד 1,לא צוין מחיר,"בקרת שמירת מרחק,גלגלי מגנזיום,היברידי",BMW 3 Series,330E Sport Hybrid OT 2.0 (184 HP)


In [44]:
df_first = pd.read_csv('yad2_car_prices_1.csv')
df_final  = pd.concat([df_first,df4])
df_final

,car_name,car_info,car_year_hand,car_price,car_tag,car_name_en,car_info_en,car_tags_en
0,ג'יפ / Jeep רנגלר ארוך,4X4 Unlimited Sahara אוט' 5 דל' 2.0 (272 כ''ס),2021 • יד 1,"300,000 ₪","גלגלי מגנזיום,טסט בתוקף,יד ראשונה",Jeep / Jeep Wrangler long,4X4 Unlimited Sahara Ot' 5 Del' 2.0 (272 HP),"Magnesium wheels, fully tested, first hand"
1,אאודי Q7,4X4 Premium חשמלי דיזל אוט' 5 מק' 3.0 (374 כ''ס),2018 • יד 2,"216,000 ₪","חסכוני,היברידי,טסט בתוקף",Audi Q7,4X4 Premium electric diesel auto 5 mk 3.0 (374...,"Economical, hybrid, valid test"
2,סובארו פורסטר,4X4 B אוט' 2.0 (150 כ''ס) [2008-2011],2011 • יד 3,"42,000 ₪","שטח קשוח,טסט בתוקף,מערכת מולטימדיה",Subaru Forester,4X4 B auto 2.0 (150 hp) [2008-2011],"Hard terrain, valid test, multimedia system"
3,ג'יפ / Jeep גרנד צ'ירוקי,4X4 Laredo אוט' 5 מק' 3.6 (286 כ''ס),2019 • יד 1,"160,000 ₪","בקרת שמירת מרחק,גלגלי מגנזיום,טסט בתוקף",Jeep / Jeep Grand Cherokee,4X4 Laredo auto 5 mk 3.6 (286 hp),"Distance control, magnesium wheels, valid test"
4,טויוטה לנד קרוזר ארוך,"4X4 STD אוט' דיזל 5 מק' 3.0 (173 כ""ס)",2008 • יד 1,"123,000 ₪","מיכל דלק גדול,טסט בתוקף,יד ראשונה",Toyota Land Cruiser long,4X4 STD OT diesel 5 mk 3.0 (173 hp),"Large fuel tank, fully tested, first hand"
...,...,...,...,...,...,...,...,...
1875,טויוטה RAV4 ארוך,Extreme אוט' 2.0 (175 כ''ס),2020 • יד 1,לא צוין מחיר,"בקרת שמירת מרחק,חסכוני,גלגלי מגנזיום",Toyota RAV4 is long,Extreme Auto 2.0 (175 hp),"Distance control, economical, magnesium wheels"
1876,אאודי A5,"Sportback Design Luxury אוט' 2.0 (150 כ""ס)",2019 • יד 2,"146,800 ₪","בקרת שיוט אדפטיבית,בקרת שמירת מרחק,גלגלי מגנזיום",Audi A5,Sportback Design Luxury Auto 2.0 (150 HP),"Adaptive cruise control, cruise control, magne..."
1877,שברולט טראקס,"LT אוט' 1.8 (140 כ""ס)",2015 • יד 2,"32,000 ₪","חסכוני,גלגלי מגנזיום,טסט בתוקף",Chevrolet Trax,LT auto 1.8 (140 hp),"Economical, magnesium wheels, fully tested"
1878,ב.מ.וו סדרה 3,330E Sport הייבריד אוט' 2.0 (184 כ''ס),2019 • יד 1,לא צוין מחיר,"בקרת שמירת מרחק,גלגלי מגנזיום,היברידי",BMW 3 Series,330E Sport Hybrid OT 2.0 (184 HP),"Distance control, magnesium wheels, hybrid"


In [45]:
df_final.to_csv('yad2_cars_data_final.csv', index=False)